In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
'I was not able to get to the ONA starter code. Hope this is what was required from me.'

In [1]:
df = pd.read_csv('medical_examination.csv')

df['overweight'] = ((df['weight'] / ((df['height'] / 100) ** 2)) > 25).astype(int)

df['cholesterol'] = (df['cholesterol'] > 1).astype(int)
df['gluc'] = (df['gluc'] > 1).astype(int)

def draw_cat_plot():
    '''
    Draw the Categorical Plot showing counts of good (0) and bad (1)
    outcomes for selected features, split by cardio in separate panels.
    Returns the matplotlib Figure object.
    '''
    df_cat = pd.melt(
        df,
        id_vars=['cardio'],
        value_vars=['cholesterol', 'gluc', 'smoke', 'alco', 'active', 'overweight']
    )

    df_cat = (
        df_cat
        .groupby(['cardio', 'variable', 'value'])
        .size()
        .reset_index(name='total')
    )

    g = sns.catplot(
        data=df_cat,
        x='variable', y='total', hue='value', col='cardio',
        kind='bar', height=5, aspect=1
    )
    g.set_axis_labels('variable', 'total')
    g._legend.set_title('value')

    fig = g.fig

    fig.savefig('catplot.png')
    return fig


def draw_heat_map():
    '''
    Draw a heatmap of the correlation matrix after cleaning the data
    for physiological plausibility and outliers. Returns the Figure.
    '''

    df_heat = df[(df['ap_lo'] <= df['ap_hi']) &
                 (df['height'] >= df['height'].quantile(0.025)) &
                 (df['height'] <= df['height'].quantile(0.975)) &
                 (df['weight'] >= df['weight'].quantile(0.025)) &
                 (df['weight'] <= df['weight'].quantile(0.975))]

    corr = df_heat.corr(numeric_only=True)

    mask = np.triu(np.ones_like(corr, dtype=bool))

    fig, ax = plt.subplots(figsize=(12, 10))

    sns.heatmap(
        corr,
        mask=mask,
        annot=True,
        fmt='.1f',
        square=True,
        center=0.0,
        linewidths=.5,
        cbar_kws={'shrink': .5}
    )

    fig.savefig('heatmap.png')
    return fig


NameError: name 'pd' is not defined

In [ ]:
if __name__ == '__main__':

    draw_cat_plot()
    draw_heat_map()